In [4]:
# Install Tesseract
%sh apt-get -f -y install tesseract-ocr 

UsageError: Line magic function `%%sh` not found.


In [ ]:
# Install other dependency
%pip install opencv-python
%pip install pytesseract
%pip install imutils

In [ ]:
# Import dependency
import cv2
import numpy as np
import pytesseract as pt
from PIL import Image
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re

## Model ##

In [ ]:
# Plot image
def plot_img(Title,img):
    
    # Define the size of the figure
    fig=plt.figure(figsize=(10,5))
    
    # Title
    plt.suptitle(Title,fontsize=20)
    
    # Show
    if type(img)!= np.ndarray:
        to_readable=mpimg.imread(img)
        plt.imshow(to_readable,cmap='gray')
    else:
        plt.imshow(img,cmap='gray')
        
    plt.axis("off")
    display(plt.show())

In [ ]:
# Haar Cascade Classifier for License plate detection
def detect_number_plate(img_path): # -> Amended
    orig=cv2.imread(img_path)
   
    # Import Cascade Classifier
    lic_data=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_russian_plate_number.xml')
   
    # Detection
    temp=orig.copy()
    plates=lic_data.detectMultiScale(temp,1.2)
    print("Number of plates detected: "+str(len(plates)))
   
    area_ls=[]
    for plate in plates:
        (x,y,w,h)=plate
        area=w*h
        area_ls.append(area)
   
    for i in range(len(plates)):
        if i == np.argmax(area_ls):
            (x,y,w,h)=plates[i]
            cv2.rectangle(temp,(x-8,y+25),(x+w-33,y+h-20),(0,255,0),3) 
    return temp

In [ ]:
# Morphological transformation
def transform(detected_img): 
   
    # 0. Resize
    ratio=detected_img.shape[0]/500
    img=imutils.resize(detected_img,height=500)
    plot_img('Resized',img) # For debug
   
    # 1. Convert to grayscale
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    plot_img('gray',gray) # For debug

    # 2. Gaussian blur
    gb=cv2.GaussianBlur(gray,(5,5),0)
    plot_img('GaussianBlur',gb) # For debug
    
    #3. Canny edges detection
    edges=cv2.Canny(gb,75,200)
    plot_img('Canny edges',edges) # For debug
    
    return edges

In [ ]:
# Masking and Cropping the plate
def masking_cropping(transformed):
   
    img=transformed.copy()
    # Contouring
    contours=cv2.findContours(img.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours=imutils.grab_contours(contours)
    contours=sorted(contours,key=cv2.contourArea, reverse=True)[:10]
   
    screenCnt=0
    for c in contours:
        # Approx the contour
        peri=cv2.arcLength(c,True)
        approx=cv2.approxPolyDP(c,0.018*peri,True)
        print('len(approx)',len(approx))
        # Assume the screen is found if our approximated contours has 4 points
        if len(approx)==4: 
            screenCnt=approx
            break

    if screenCnt is None:
        detected=0
        print("No contour detected")

    else:
        detected=1

    if detected==1:
        cv2.drawContours(img,[screenCnt],-1,(0,0,255),3)
       
    # Masking
    mask=np.zeros(img.shape,np.uint8)
    new_img=cv2.drawContours(mask,[screenCnt],0,255,-1,)
    new_img=cv2.bitwise_and(img,img,mask=mask)
    plot_img('masked',new_img) # For debug
    
    # Cropping
    (x,y)=np.where(mask==255)
    (topx,topy)=(np.min(x),np.min(y))
    (bottomx,bottomy)=(np.max(x),np.max(y))
    Cropped=new_img[topx:bottomx+1,topy:bottomy+1]
    plot_img('cropped',Cropped) # For debug
    return Cropped

In [ ]:
# Accuracy calculation
def cal_accuracy(pred,actual):
    # Initialise
    accuracy="0 %"
    num_matches=0
    
    # all matches
    if pred==actual:
        accuracy='100 %'
        
    else:
        # same length
        #if len(pred)==len(actual):
        for p,a in zip(pred,actual):
            if p==a:
                num_matches+=1
        accuracy=str(round(num_matches/len(actual),2)*100)
        accuracy+= "%"
    return accuracy

In [ ]:
# Combine all the aboe funcitons
def master(plate,actual):
    detected=detect_number_plate(plate)
    plot_img('detected',detected)
    
    transformed=transform(detected)
    plot_img('transformed',transformed)
    
    maskedcropped=masking_cropping(transformed)
    plot_img('Cropped_plate',maskedcropped)
    plate_text_maskedcropped=pt.image_to_string(maskedcropped).strip()
    print('Detected text: ',plate_text_maskedcropped)
    print('Accuracy: ',cal_accuracy(plate_text_maskedcropped,actual))